# Linear Regression

In this part of the project, we are going to examine the linear regression analysis of our dataset. First, we need to import the necessary packages and datasets.


In [1]:
#%pip install sklego # run when necessary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 7.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklego.linear_model import LADRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import  Ridge
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.impute import SimpleImputer

df_train = pd.read_csv('/content/train_df_doordash_pre_imp_scaling.csv')
df_test = pd.read_csv('/content/test_df_doordash_pre_imp_scaling.csv')

In [3]:
df_train.head()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,...,market_id_5,market_id_6,order_protocol_1,order_protocol_2,order_protocol_3,order_protocol_4,order_protocol_5,order_protocol_6,order_protocol_7,created_weekend
0,1,760,1,760,760,20.0,20.0,21.0,251,810.0,...,0,0,0,0,0,1,0,0,0,1
1,2,2266,1,856,949,54.0,40.0,64.0,251,525.0,...,0,0,0,1,0,0,0,0,0,0
2,5,3585,4,150,1295,22.0,14.0,22.0,251,216.0,...,1,0,0,0,0,0,1,0,0,0
3,1,1050,1,1050,1050,15.0,14.0,16.0,251,773.0,...,0,0,0,1,0,0,0,0,0,0
4,4,3635,3,450,1825,5.0,3.0,3.0,446,236.0,...,0,0,1,0,0,0,0,0,0,0


In [4]:
df_test.head()

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,...,market_id_5,market_id_6,order_protocol_1,order_protocol_2,order_protocol_3,order_protocol_4,order_protocol_5,order_protocol_6,order_protocol_7,created_weekend
0,6,6235,6,495,1425,37.0,29.0,59.0,251,39.0,...,0,0,0,1,0,0,0,0,0,0
1,7,6150,5,500,1250,51.0,51.0,103.0,251,608.0,...,0,0,0,0,0,0,1,0,0,0
2,3,1790,3,500,650,42.0,43.0,63.0,251,847.0,...,0,0,0,0,0,1,0,0,0,1
3,1,1000,1,1000,1000,37.0,36.0,46.0,251,691.0,...,0,0,0,1,0,0,0,0,0,1
4,1,1200,1,1200,1200,NaN,NaN,NaN,251,682.0,...,0,1,0,1,0,0,0,0,0,0


For our response variable in this model, we are choosing delivery_duration_min, as we are trying to predict the amount of time it takes for a delivery. This will help later on when we will predict whether an order is considered "slow" or not using various classification methods.

In order to identify predictors, we will look at the correlations of the variables with delivery_duration_min. We can take the variables with the highest absolute value correlations and bring them into our linear model:

In [5]:
corr = df_train.corr(numeric_only=True)
print(corr['delivery_duration_min'].sort_values(ascending=False))

delivery_duration_min                           1.000000
slow_order                                      0.779382
estimated_store_to_consumer_driving_duration    0.239718
subtotal                                        0.220960
total_outstanding_orders                        0.191451
num_distinct_items                              0.161011
max_item_price                                  0.135084
total_items                                     0.119414
estimated_order_place_duration                  0.104651
total_busy_dashers                              0.097397
market_id_1                                     0.096175
order_protocol_1                                0.078363
total_onshift_dashers                           0.076827
created_weekend                                 0.066264
order_protocol_6                                0.043873
min_item_price                                  0.012595
order_protocol_4                                0.010192
market_id_3                    

From here, we can see that the estimated_store_to_consumer_driving_duration is the best predictor (highest correlation to our response variable). So, we will use this for our regression model, as well as the top 10 highest correlated variables. We cannot use slow_order because that was engineered directly from delivery_duration_min and therefore would just be redundant and incorrect to use.

To form our regression model, the code is shown below. We first need to apply imputation, though, via a pipeline:

In [16]:
pipe = Pipeline([
    ("imputer",SimpleImputer(strategy = "median")),
    ("scaler", StandardScaler()),
    ("linear_model", LinearRegression())
])

In [9]:
# splitting data
# top 10 most correlated variables
predictors = ["estimated_store_to_consumer_driving_duration", "subtotal", "total_outstanding_orders",
              "num_distinct_items", "max_item_price", "total_items", "estimated_order_place_duration",
              "market_id_1", "total_busy_dashers", "created_hour"]
# tts
X = df_train[predictors]
X_test = df_test[predictors]
y = df_train['delivery_duration_min']
y_test = df_test['delivery_duration_min']


In [17]:
# running multiple linear regression
pipe.fit(X, y)
linear_model = pipe.named_steps["linear_model"]

print("Intercept:", linear_model.intercept_)
print("Coefficients:", linear_model.coef_)

Intercept: 47.49039793591896
Coefficients: [  4.23898675   2.43780192  12.42608857   0.57630038   0.32829239
   0.13589789   1.7451748    2.05064429 -10.10751342  -1.48086757]


We can see the accuracy metrics of our model below:

In [18]:
# model evaluation pipeline
def model_evaluation(model, X, y):
  y_pred = model.predict(X)
  r2 = r2_score(y, y_pred)
  rMSE = np.sqrt(mean_squared_error(y, y_pred))
  MAE = mean_absolute_error(y, y_pred)
  MAD = np.median(np.abs(y_pred - y))
  corr = np.corrcoef(y, y_pred)[0, 1]
  return {
      'R^2': r2,
      'rMSE': rMSE,
      'MAE': MAE,
      'MAD': MAD,
      'r': corr
  }

# ls model results
print(model_evaluation(pipe, X, y))
print(model_evaluation(pipe, X_test, y_test))

{'R^2': 0.2053888313780451, 'rMSE': np.float64(15.947358311263846), 'MAE': 11.858568122935537, 'MAD': np.float64(9.661677206216737), 'r': np.float64(0.45319844591309505)}
{'R^2': 0.197184724675453, 'rMSE': np.float64(16.15428626106897), 'MAE': 12.003809265553642, 'MAD': np.float64(9.792645338465132), 'r': np.float64(0.44407681894378104)}


As shown above, we have the accuracy metrics for our LS model, including a 0.2 R^2, rMSE of 16, and MAE of 12 (all rounded) for both training and testing. This means there is no over/under fitting, however this is by no means the most accurate model, only accounting for about 20% of the data's variance and shape.

We can also use LAD regression instead of just LS, although it appears to be less accurate, as seen in the lower R^2 and higher rMSE (but it does have a better MAD). The code for this, including results, is shown below:

In [19]:
# lad model
pipe_lad = Pipeline([
    ("imputer",SimpleImputer(strategy = "median")),
    ("scaler", StandardScaler()),
    ("linear_model", LADRegression())
])
pipe_lad.fit(X, y)
linear_model = pipe_lad.named_steps["linear_model"]
print("Intercept:", linear_model.intercept_)
print("Coefficients:", linear_model.coef_)

# lad model results
print(model_evaluation(pipe_lad, X, y))
print(model_evaluation(pipe_lad, X_test, y_test))

/usr/local/lib/python3.12/dist-packages/sklego/linear_model.py:1358: UserWarning: Please consider using scikit-learn version of quantile regression.

Hint: `from sklearn.linear_model import QuantileRegressor`
Docs: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.QuantileRegressor.html
  warn(msg, UserWarning)


Intercept: 44.43427265426853
Coefficients: [ 4.14898787  2.53174687 10.97148866  0.53326584  0.27920555  0.15834722
  1.68814078  1.72448543 -8.16809374 -1.16538109]
{'R^2': 0.1735144165600888, 'rMSE': np.float64(16.264063489976117), 'MAE': 11.580193160565166, 'MAD': np.float64(8.757938007888036), 'r': np.float64(0.45058670449031646)}
{'R^2': 0.16543898010676272, 'rMSE': np.float64(16.470584402911953), 'MAE': 11.718571077562023, 'MAD': np.float64(8.855765043859293), 'r': np.float64(0.4420734650346507)}


We can see that our accuracy is not the best, with an R^2 of 0.2 being the best between the LS and LAD models.

However, instead of just using the top 10 predictors, we could simply use all of them. The R^2 will be guaranteed to increase, but it makes the model more complicated, less interpretable, and it risks overfitting. We can see if it is worth taking these risks if the accuracy is high enough:

In [21]:
# instead of choosing the top variables, what if we did all of the variables?

# redefining X
X_train = df_train.drop(columns=['delivery_duration_min', 'slow_order'])
X_test = df_test.drop(columns=['delivery_duration_min', 'slow_order'])

# ls model and results
pipe.fit(X_train, y)
linear_model = pipe.named_steps["linear_model"]
print("Intercept:", linear_model.intercept_)
print("Coefficients:", linear_model.coef_)
print(model_evaluation(pipe, X_train, y))
print(model_evaluation(pipe, X_test, y_test))

# lad model and results
pipe_lad.fit(X_train, y)
linear_model = pipe_lad.named_steps["linear_model"]
print("Intercept:", linear_model.intercept_)
print("Coefficients:", linear_model.coef_)
print(model_evaluation(pipe_lad, X_train, y))
print(model_evaluation(pipe_lad, X_test, y_test))


Intercept: 47.49039793591896
Coefficients: [-1.70003824e-01  2.79748227e+00  5.89751292e-01  5.68087101e-02
  3.85788992e-01 -1.15723484e+01 -3.56984418e+00  1.74427522e+01
  2.08428466e+00  4.30469711e+00 -1.63279825e+00 -1.24053921e-01
  1.35804901e+00 -8.28400162e-01  2.07022300e-01 -6.68673180e-01
 -2.18908630e-01  4.54849632e-01 -6.04670903e-01  3.14524548e-01
 -1.09631621e-01  1.13970601e+00 -3.86143814e-01  5.89166751e-01
 -9.30190847e-03  4.01547819e-01]
{'R^2': 0.25306972217074, 'rMSE': np.float64(15.461493848869665), 'MAE': 11.445096520752113, 'MAD': np.float64(9.227309930069367), 'r': np.float64(0.5030603563895086)}
{'R^2': 0.2467831485805233, 'rMSE': np.float64(15.647320375045584), 'MAE': 11.559139504753078, 'MAD': np.float64(9.28396994623591), 'r': np.float64(0.4967885667154489)}
Intercept: 44.704943882904104
Coefficients: [-0.26144091  2.84017277  0.63103981  0.02466505  0.34840836 -9.82267051
 -2.79578485 15.56281912  1.77076564  4.18128797 -1.36085493 -0.07477275
  1.12

Well, it is not better to do all of the predictors. The R^2 only increased about +0.05 with all variables instead of just the top 10, and the rMSE, MAE, and MAD all stayed about the same. The disadvantages of using too many predictors outweight the advantages, so we will continue to use the top 10 predictors.

Finally, as per the constraints of the original project check in, we used Ridge Regression in hopes of increasing accuracy and stability. The code for such regularization is shown below:

In [22]:
#use pipline to fit scalar quickly
pipe = Pipeline([
    ("imputer",SimpleImputer(strategy = "median")),
    ("scaler", StandardScaler()),
    ("ridge", Ridge(random_state=0))
])

# define alphas and cross validation
alphas = np.logspace(-4, 4, 30)
cv = KFold(n_splits=10, shuffle=True, random_state=0)

# cross validation
g = GridSearchCV(
    pipe,
    param_grid={"ridge__alpha": alphas},
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1
)
g.fit(X, y)

best = g.best_estimator_
X_test = df_test[predictors]
y_pred_val = best.predict(X_test)

# accuracy metrics for ridge regression
print("Best alpha:", g.best_params_["ridge__alpha"])
print("Val MSE:", mean_squared_error(y_test, y_pred_val))
print("Val R^2:", r2_score(y_test, y_pred_val))

Best alpha: 0.7278953843983146
Val MSE: 260.96086138537396
Val R^2: 0.1971850422169702


In [ ]:
np.sqrt(260.960861385374)

np.float64(16.154283066276076)

As we can see, the R^2 did not really increase at all, and the rMSE similarly did not change at all (both hovering around 0.2 and 16.1, respectively) afer ridge. Therefore, ridge regression did not help much in the overall shape and accuracy of the model.

Overall, after running multiple linear regression models using Least Squares, Least Absolute Deviation, and Ridge Regression, we can see fairly dismal results. With a peak R^2 of only about 0.2, the model does not account for a lot of the actual data. This is probably due to the lower correlations between variables, as none had higher than 0.25 correlation with delivery_duration_min. At least there was no overfitting or underfitting, as the results were fairly consistent between train and testing sets. This also helps us recognize that the data is probably not linear, and should be addressed as such in the future. Moreover, given that we are ultimately trying to solve a classification problem of if an order will be slow or not, it is okay that linear regression is not the most accurate. Next, we can move on to logistic regression.